In [218]:
import pandas as pd
from sqlalchemy import create_engine
import re
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()

## Get 5 years of player salaries

In [219]:
player_data_df = pd.DataFrame(columns=['club', 'last_name','first_name','position','base_salary','guaranteed_compensation'])
year_range = np.arange(2013,2018)

for year in year_range:
    csv = f"mls-salaries-{year}.csv"
    
    player_info_year = pd.read_csv(csv)
    player_info_year["year"] = year
    
    player_data_df = player_data_df.append(player_info_year,sort=False)

player_data_df.head()

,club,last_name,first_name,position,base_salary,guaranteed_compensation,year
0,VAN,Abdallah,Aminu,M,46500.0,46500.00,2013.0
1,DAL,Acosta,Kellyn,M,46500.0,70500.00,2013.0
2,VAN,Adekugbe,Samuel,D,46500.0,51500.00,2013.0
3,TOR,Agbossoumonde,Gale,D,46500.0,53166.67,2013.0
4,NE,Agudelo,Juan,F,145000.0,175000.00,2013.0


## Create players table

In [220]:
players_grouped = player_data_df.groupby(['last_name','first_name'])
players_years = players_grouped["year"].count()

players_df = pd.DataFrame({'years_played':players_years})
players_df.reset_index(inplace=True)
del players_df['years_played']

players_df.reset_index(inplace=True)
players_df.rename(columns={'index':'id'},inplace=True)

players_df.head()

,id,last_name,first_name
0,0,Abang,Anatole
1,1,Abdallah,Aminu
2,2,Abdul-Salaam,Saad
3,3,Abu,Mohammed
4,4,Abubakar,"Alhassan ""Lalas"""


## Create player salary table linked to players table

In [221]:
player_info_df = player_data_df.loc[:,['year', 'club','position', 'base_salary', 'guaranteed_compensation', 'last_name','first_name']]
player_info_df.rename(columns={'club':'team_id'},inplace=True)
player_info_df.head()

,year,team_id,position,base_salary,guaranteed_compensation,last_name,first_name
0,2013.0,VAN,M,46500.0,46500.00,Abdallah,Aminu
1,2013.0,DAL,M,46500.0,70500.00,Acosta,Kellyn
2,2013.0,VAN,D,46500.0,51500.00,Adekugbe,Samuel
3,2013.0,TOR,D,46500.0,53166.67,Agbossoumonde,Gale
4,2013.0,NE,F,145000.0,175000.00,Agudelo,Juan


In [222]:
player_info_df = pd.merge(player_info_df,players_df,left_on=['last_name','first_name'],right_on=['last_name','first_name'])
player_info_df.head()

,year,team_id,position,base_salary,guaranteed_compensation,last_name,first_name,id
0,2013.0,VAN,M,46500.0,46500.0,Abdallah,Aminu,1
1,2013.0,DAL,M,46500.0,70500.0,Acosta,Kellyn,7
2,2014.0,DAL,M,55000.0,79000.0,Acosta,Kellyn,7
3,2015.0,DAL,M,60000.0,84000.0,Acosta,Kellyn,7
4,2016.0,DAL,M-D,220000.0,240000.0,Acosta,Kellyn,7


In [223]:
player_info_df.reset_index(inplace=True)
player_info_df.rename(columns={'index':'id', 'id':'player_id'},inplace=True)
del player_info_df['last_name']
del player_info_df['first_name']
player_info_df.head()

,id,year,team_id,position,base_salary,guaranteed_compensation,player_id
0,0,2013.0,VAN,M,46500.0,46500.0,1
1,1,2013.0,DAL,M,46500.0,70500.0,7
2,2,2014.0,DAL,M,55000.0,79000.0,7
3,3,2015.0,DAL,M,60000.0,84000.0,7
4,4,2016.0,DAL,M-D,220000.0,240000.0,7


## Create teams table from scraped web data

In [224]:
url = 'https://www.mlssoccer.com/stats/team'
all_tables = pd.read_html(url)
scrape_dt = datetime.now()

mls_league = all_tables[0]
mls_league.columns = ["team_name", "games_played", "goals", "assists", "shots", "shots_on_goal", "fouls_committed", "fouls_suffered", \
                     "offsides", "corner_kicks", "penalty_goal_kicks", "penalty_kick_attempts"]
mls_league.set_index("team_name", inplace=True)
mls_league["scrape_date"] = scrape_dt

team_codes = pd.read_excel("team_codes.xlsx")

mls_league = team_codes.merge(mls_league, left_on=["team_name"], right_on=["team_name"])
mls_league.rename(columns={'team_code':'id'},inplace=True)


mls_league.head()

,team_name,id,games_played,goals,assists,shots,shots_on_goal,fouls_committed,fouls_suffered,offsides,corner_kicks,penalty_goal_kicks,penalty_kick_attempts,scrape_date
0,Atlanta United,ATL,34,70,63,511,203,346,399,77,173,12,14,2018-11-14 20:35:09.686323
1,Sporting Kansas City,KC,34,65,69,583,207,376,377,38,206,5,7,2018-11-14 20:35:09.686323
2,Seattle Sounders FC,SEA,34,52,57,430,135,357,430,69,183,5,5,2018-11-14 20:35:09.686323
3,Los Angeles FC,LAFC,34,68,64,527,204,361,468,52,203,4,5,2018-11-14 20:35:09.686323
4,FC Dallas,DAL,34,52,50,483,177,351,382,79,189,4,6,2018-11-14 20:35:09.686323


## Only keep player data for players on current MLS teams

In [242]:
mls_teams = mls_league['id'].unique()

player_info_df['curr_team'] = ''

for index, row in player_info_df.iterrows():
    if row[2] in mls_teams:
        player_info_df.loc[index,'curr_team'] = 'Y'
    else:
        player_info_df.loc[index,'curr_team'] = 'N'

player_info_df = player_info_df.loc[(player_info_df['curr_team']=='Y'),:]

del player_info_df['curr_team']

## Load tables into MySQL

In [246]:
rds_connection_string = "root:password@127.0.0.1/mls_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [247]:
engine.table_names()

['player_info', 'players', 'team_stats']

In [248]:
players_df.to_sql(name='players', con=engine, if_exists='append', index=False)

In [249]:
mls_league.to_sql(name='team_stats', con=engine, if_exists='append', index=False)

In [250]:
player_info_df.to_sql(name='player_info', con=engine, if_exists='append', index=False)